In [3]:
from gplearn.genetic import SymbolicTransformer
from sklearn.utils import check_random_state
from sklearn.datasets import load_diabetes
from sklearn.linear_model import Ridge
import numpy as np

In [4]:
rng = check_random_state(0)
diabetes = load_diabetes() #糖尿病数据库，回归任务
perm = rng.permutation(diabetes.target.size)
diabetes.data = diabetes.data[perm]
diabetes.target = diabetes.target[perm]

#先用ridge回归器训练一下
est = Ridge()
est.fit(diabetes.data[:300, :], diabetes.target[:300]) #前300个数据集
print(est.score(diabetes.data[300:, :], diabetes.target[300:]))

0.43405742105789413


In [5]:
function_set = ['add', 'sub', 'mul', 'div',
                'sqrt', 'log', 'abs', 'neg', 'inv',
                'max', 'min']
gp = SymbolicTransformer(generations=20, population_size=2000,
                         hall_of_fame=100, n_components=10,
                         function_set=function_set,
                         parsimony_coefficient=0.0005,
                         max_samples=0.9, verbose=1,
                         random_state=0, n_jobs=3)
gp.fit(diabetes.data[:300, :], diabetes.target[:300]) #先训练一下，这里会生成新的特征，而不是完整的模型

    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0    11.37         0.126618        5         0.612827             0.68      1.30m
   1     6.63         0.344375        3         0.659086         0.451797     25.55s
   2     5.36         0.473731        3         0.669019         0.321485     26.03s
   3     4.74         0.587613        3         0.673354          0.31249     23.20s
   4     4.39         0.597151       13         0.675282          0.49482     23.04s
   5     4.31         0.611995       15         0.686134         0.148203     20.53s
   6     4.89         0.611891        9         0.685649         0.199002     19.07s
   7     6.49         0.617031        9         0.688309         0.287286     19.75s
   8     9.19         0.628618       17         0.720606          0.26362  

SymbolicTransformer(function_set=['add', 'sub', 'mul', 'div', 'sqrt', 'log',
                                  'abs', 'neg', 'inv', 'max', 'min'],
                    max_samples=0.9, n_jobs=3, parsimony_coefficient=0.0005,
                    population_size=2000, random_state=0, verbose=1)

In [6]:
gp_features = gp.transform(diabetes.data)  #得到新特征
new_diabetes = np.hstack((diabetes.data, gp_features))  #在原特征基础上加上新特征   hstack将数组进行水平合并  
print(new_diabetes.shape)
print(diabetes.shape)

(442, 20)
(442, 10)


In [7]:
est = Ridge()
est.fit(new_diabetes[:300, :], diabetes.target[:300])
print(est.score(new_diabetes[300:, :], diabetes.target[300:]))

0.5336788517320443
